In [1]:
import pickle as pkl
import numpy as np
import nltk
import os
from nltk.tokenize import word_tokenize
import copy

In [2]:
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')

In [3]:
file_list = ['bbc/' + folder_name + '/' + file_name for folder_name in sorted(os.listdir('bbc'))[1:] for file_name in os.listdir('bbc/' + folder_name)[1:]]

In [4]:
file_list[:10]

['bbc/business/001.txt',
 'bbc/business/002.txt',
 'bbc/business/003.txt',
 'bbc/business/004.txt',
 'bbc/business/005.txt',
 'bbc/business/006.txt',
 'bbc/business/007.txt',
 'bbc/business/008.txt',
 'bbc/business/009.txt',
 'bbc/business/010.txt']

In [5]:
sentence_list = []
for file in file_list:
    with open('./' + file, 'r', encoding = 'utf-8') as fr:
        for i,line in enumerate(fr.readlines()):
            if line.strip()!=' ':
                for l in sent_detector.tokenize(line.strip()):
                    split_sentence = word_tokenize(l.lower())
                    if len(split_sentence) >= 4 and len(split_sentence) <= 30:
                        sentence_list.append(split_sentence)
            else:
                continue

In [6]:
correct_sentence_list = copy.deepcopy(sentence_list)
len(correct_sentence_list)

36198

In [7]:
import pickle as pkl
import random
with open('cor2err_vocab.pkl', 'rb') as f:
    chr2err = pkl.load(f)

In [8]:

for i, sent in enumerate(sentence_list):
    sent_idx = [idx for idx in range(len(sent))]
    # 문장 내 토큰의 최대 60%의 토큰만 변환
    # 예) '나는' '오늘' '너무' '학교에' '가고' '싶은데,' '태풍' '때문에' '가지' '못한다.''
    #     라는 문장에서 최대 6개의 토큰에만 노이즈를 생성함.
    k_value = random.choice([idx for idx in range(1, int(len(sent_idx) * 0.6)) ])
    idx_list = sorted(random.sample(sent_idx, k = k_value))
    for idx in reversed(idx_list):
        if sent[idx] in list(chr2err.keys()):
            sentence_list[i][idx] = random.choice(chr2err[sent[idx]])
            print(i,idx, end = '\t' )
        else:
            # 토큰이 미리 정의된 MS error vocab 에 정의 되지 않았다면, 
            # 30% 확률로 해당 토큰을 문장에서 제거
            random_value = random.choice([i for i in range(1, 11)])
            if random_value < 4:
                del sentence_list[i][idx]
            else:
                continue
    break

0 2	

In [9]:
print(sentence_list[i])
print(correct_sentence_list[i])

['dollar', 'gains', 'con,', 'greenspan', 'speech']
['dollar', 'gains', 'on', 'greenspan', 'speech']


In [10]:
with open('./output_preprocesscorrect_sentence_listput.pkl', 'wb') as f:
    pkl.dump(sentence_list, f)
with open('./output_preprocess/' + 'bbc_output.pkl', 'wb') as f:
    pkl.dump(correct_sentence_list, f)

In [11]:
import pandas as pd
import re
from tqdm import tqdm

In [12]:
content = pd.read_csv('./1974_3493_bundle_archive/articles1.csv').iloc[:, -1].to_list()
content.extend(pd.read_csv('./1974_3493_bundle_archive/articles2.csv').iloc[:, -1].to_list())
content.extend(pd.read_csv('./1974_3493_bundle_archive/articles3.csv').iloc[:, -1].to_list())

In [13]:
sentence_list = []
for i, cont in enumerate(tqdm(content)):
#     cont = re.sub('[|]')
    for sent in sent_detector.tokenize(cont.strip().lower()):
        split_sent = word_tokenize(sent)
        if len(split_sent) < 4:
            continue
        if len(split_sent) >= 4 and len(split_sent) <= 30:
            sentence_list.append(split_sent)

100%|█████████████████████████████████████████████████████████████████████████| 142570/142570 [14:16<00:00, 166.48it/s]


In [14]:
correct_sentence_list = copy.deepcopy(sentence_list)
len(correct_sentence_list)

3349657

In [15]:
for i, sent in enumerate(tqdm(sentence_list)):
    sent_idx = [j for j in range(len(sent))]
    # 문장 내 토큰의 최대 60%의 토큰만 변환
    # 예) '나는' '오늘' '너무' '학교에' '가고' '싶은데,' '태풍' '때문에' '가지' '못한다.''
    #     라는 문장에서 최대 6개의 토큰에만 노이즈를 생성함.
#     print(i, len(sent_idx))
    if len(sent_idx) < 4:
        print(i)
    k_value = random.choice([j for j in range(1, int(len(sent_idx) * 0.6)) ])
    idx_list = sorted(random.sample(sent_idx, k = k_value))
    for idx in reversed(idx_list):
        if sent[idx] in list(chr2err.keys()):
            sentence_list[i][idx] = random.choice(chr2err[sent[idx]])
        else:
            # 토큰이 미리 정의된 MS error vocab 에 정의 되지 않았다면, 
            # 30% 확률로 해당 토큰을 문장에서 제거
            random_value = random.choice([i for i in range(1, 11)])
            if random_value < 4:
                del sentence_list[i][idx]
            else:
                continue

100%|██████████████████████████████████████████████████████████████████████| 3349657/3349657 [33:14<00:00, 1679.37it/s]


In [16]:
with open('./output_preprocess/' + 'kaggle_input.pkl', 'wb') as f:
    pkl.dump(sentence_list, f)
with open('./output_preprocess/' + 'kaggle_output.pkl', 'wb') as f:
    pkl.dump(correct_sentence_list, f)

In [17]:
sentence_list[0]

['washington',
 '—',
 'congressional',
 'republicans',
 'have',
 'a',
 'mews',
 'fear',
 'when',
 'it',
 'comes',
 'the',
 'thieir.',
 'health',
 'lawsuit',
 'thet',
 'obama',
 ':',
 "lethey'",
 'dighty',
 'win',
 '.']

In [18]:
import random
a = [1,2,3,4,5,6,7,8,9]
random.shuffle(a)

In [19]:
print(a)

[1, 4, 5, 8, 7, 6, 2, 9, 3]
